### Questão 7: Escolher um dataset e aplicar o algoritmo Deep convolutional GANs (DCGAN)

#### Dataset escolhido: https://www.kaggle.com/datasets/ash2703/handsignimages

In [6]:
import tensorflow as tf
import PIL

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'hand-sign-dataset/Train',
    labels='inferred',
    label_mode='int',
    color_mode='grayscale'
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'hand-sign-dataset/Test',
    labels='inferred',
    label_mode='int',
    color_mode='grayscale'
)

Found 27455 files belonging to 24 classes.
Found 7172 files belonging to 24 classes.


In [8]:
print(train_ds.class_names)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']
